In [1]:
import MC_functions as mc
import numpy as np
import pandas as pd
from scipy.stats import multivariate_t
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
import rpy2.robjects.numpy2ri
import multiprocessing as mp
import concurrent.futures
import matplotlib.pyplot as plt

## Important note: make sure rpy2 is version 3.5.1 or below

In [12]:
#! pip install rpy2==3.5.1

In [2]:
# Simulation parameters
number_of_variables = 1
beta = np.array([2]) # in the multivariate case, just add elements until the size of the vector is equal to number_of_variables
N1 = 5
N2 = 10
T = 3
n = N1 * N2 * T
case = 2 # correspond to the three cases of omega in the chapter
mu_epsilon = 0
sigma_epsilon = 8
non_zero_prob = 0.6
mu_x = 0
sigma_x = 1
specification = "normal" # "normal", "t" or "sn"
mu_e = 0
mu_e_vec= np.full(n, mu_e)
sigma_e = 1
t_dist_degree = 5
mu_U = 0
lambda_parameter = 2
spcov = importr("spcov")
ksi_1 = 18  # controls the smallest penalty, the higher the value the smaller 
ksi_2 = 5   # flattening parameter for sigmoid penalty function
seed = 42
np.random.seed(seed)

In [3]:
rpy2.robjects.numpy2ri.activate()

In [4]:
# Step 1: Generate x variables
x = np.random.normal(mu_x, sigma_x, (n, number_of_variables))

# Step 2: Generate three fixed-effect variables
fixed_effect_1 = np.random.normal(-2, 1, (N1, 1))
fixed_effect_2 = np.random.normal(3, 1, (N2, 1))
fixed_effect_3 = np.random.normal(1, 1, (T, 1))

# Step 3: Repeat the fixed-effect variables to match the desired dimensions
fixed_effect_1 = np.repeat(fixed_effect_1, N2 * T, axis=0)
fixed_effect_2 = np.tile(fixed_effect_2, (N1 * T, 1))
fixed_effect_3 = np.tile(fixed_effect_3, (N1 * N2, 1))

# Step 4: Stack the fixed-effect variables and combine x and fixed effects
fixed_effects_matrix = np.hstack((fixed_effect_1, fixed_effect_2, fixed_effect_3))
X = np.hstack((x, fixed_effects_matrix))

# Step 5: generate distrubances using the covariance matrix
omega = mc.generate_omega(case, mu_e, sigma_e, non_zero_prob, N1, N2, T, seed)
disturbances = mc.generate_disturbances(mu_e_vec, omega, n, t_dist_degree, lambda_parameter, mu_U, specification, seed)

# Step 6: Generate y using linear relationship: y = X * beta + FE + disturbances
y = np.dot(x, beta) + fixed_effects_matrix.sum(axis=1) + disturbances.flatten()

# Step 7: Transform X and y
x_tilde, y_tilde = mc.transform_xy(X, y,number_of_variables)
X = np.hstack((x_tilde, fixed_effects_matrix))
y = y_tilde

# Step 8: Estimate beta using OLS formula
X_with_intercept = np.hstack((np.ones((n, 1)), X))  # Adding intercept term
X_transpose = np.transpose(X_with_intercept)
X_transpose_X_inv = np.linalg.inv(np.dot(X_transpose, X_with_intercept))
beta_hat = np.dot(np.dot(X_transpose_X_inv, X_transpose), y)

# Step 9: Calculate the variance-covariance matrix of beta_hat
# OLS
epsilon_hat = y - np.dot(X_with_intercept, beta_hat)
omega_hat = np.dot(epsilon_hat, epsilon_hat)/(n-number_of_variables-4)
sigma_hat = np.dot(np.dot(np.dot(np.dot(X_transpose_X_inv,X_transpose),omega_hat), X_with_intercept),X_transpose_X_inv)

# robust
omega_hat_robust = np.outer(epsilon_hat, epsilon_hat)
omega_hat_robust = np.eye(omega_hat_robust.shape[0]) * omega_hat_robust
sigma_hat_robust = np.dot(np.dot(np.dot(np.dot(X_transpose_X_inv,X_transpose),omega_hat_robust), X_with_intercept),X_transpose_X_inv)

# spcov
init_omega = omega_hat_robust
Lambda = mc.generate_penalty_matrix(n, ksi_1,ksi_2)
S = mc.make_positive_definite(np.outer(epsilon_hat, epsilon_hat))
init_omega_r = ro.r.matrix(init_omega, nrow=n, ncol=n)
S_r = ro.r.matrix(S, nrow=n, ncol=n)
Lambda_r = ro.r.matrix(Lambda, nrow=n, ncol=n)
spcov_result = spcov.spcov(
init_omega_r,
S_r,
Lambda_r,
step_size=0.1,
nesterov = True,
n_outer_steps = 100,
n_inner_steps = 100,
tol_outer = 1e-04,
thr_inner = 0.01,
backtracking = 0.2,
trace = 0
)

omega_hat_spcov = spcov_result[1]

# Step 10: Construct the t-statistics for beta_hat
t_stat_robust = beta_hat / np.sqrt(np.diag(sigma_hat_robust))


In [5]:
# Initialize the R context for each thread or process
def initialize_r_context():
    import rpy2.robjects as ro
    from rpy2.robjects.packages import importr
    import rpy2.robjects.numpy2ri
    
    # Activate numpy2ri conversion within the local R context
    rpy2.robjects.numpy2ri.activate()    
    spcov = importr("spcov")
    

In [13]:
def run_iteration(i):
    # Initialize R context for this thread or process
    initialize_r_context()
    print(f"Iteration {i + 1} started...")
    try: 
        # Step 1: Generate x variables
        x = np.random.normal(mu_x, sigma_x, (n, number_of_variables))

        # Step 2: Generate three fixed-effect variables
        fixed_effect_1 = np.random.normal(-2, 1, (N1, 1))
        fixed_effect_2 = np.random.normal(3, 1, (N2, 1))
        fixed_effect_3 = np.random.normal(1, 1, (T, 1))

        # Step 3: Repeat the fixed-effect variables to match the desired dimensions
        fixed_effect_1 = np.repeat(fixed_effect_1, N2 * T, axis=0)
        fixed_effect_2 = np.tile(fixed_effect_2, (N1 * T, 1))
        fixed_effect_3 = np.tile(fixed_effect_3, (N1 * N2, 1))

        # Step 4: Stack the fixed-effect variables and combine x and fixed effects
        fixed_effects_matrix = np.hstack((fixed_effect_1, fixed_effect_2, fixed_effect_3))
        X = np.hstack((x, fixed_effects_matrix))

        # Step 5: generate distrubances using the covariance matrix
        omega = mc.generate_omega(case, mu_e, sigma_e, non_zero_prob, N1, N2, T, seed)
        disturbances = mc.generate_disturbances(mu_e_vec, omega, n, t_dist_degree, lambda_parameter, mu_U, specification, seed)

        # Step 6: Generate y using linear relationship: y = X * beta + FE + disturbances
        y = np.dot(x, beta) + fixed_effects_matrix.sum(axis=1) + disturbances.flatten()

        # Step 7: Transform X and y
        x_tilde, y_tilde = mc.transform_xy(X, y,number_of_variables)
        X = np.hstack((x_tilde, fixed_effects_matrix))
        y = y_tilde

        # Step 8: Estimate beta using OLS formula
        X_with_intercept = np.hstack((np.ones((n, 1)), X))  # Adding intercept term
        X_transpose = np.transpose(X_with_intercept)
        X_transpose_X_inv = np.linalg.inv(np.dot(X_transpose, X_with_intercept))
        beta_hat = np.dot(np.dot(X_transpose_X_inv, X_transpose), y)

        # Step 9: Calculate the variance-covariance matrix of beta_hat

        #OLS
        epsilon_hat = y - np.dot(X_with_intercept, beta_hat)
        omega_hat_ols = np.dot(epsilon_hat, epsilon_hat)/(n-number_of_variables-4)
        sigma_hat_ols = np.dot(np.dot(np.dot(np.dot(X_transpose_X_inv,X_transpose),omega_hat_ols), X_with_intercept),X_transpose_X_inv)

        #Robust
        omega_hat_robust = np.outer(epsilon_hat, epsilon_hat)
        omega_hat_robust = np.eye(omega_hat_robust.shape[0]) * omega_hat_robust
        sigma_hat_robust = np.dot(np.dot(np.dot(np.dot(X_transpose_X_inv,X_transpose),omega_hat_robust), X_with_intercept),X_transpose_X_inv)

        #spcov
        init_omega = omega_hat_robust
        Lambda = mc.generate_penalty_matrix(n, ksi_1,ksi_2)
        S = mc.make_positive_definite(np.outer(epsilon_hat, epsilon_hat))
        init_omega_r = ro.r.matrix(init_omega, nrow=n, ncol=n)
        S_r = ro.r.matrix(S, nrow=n, ncol=n)
        Lambda_r = ro.r.matrix(Lambda, nrow=n, ncol=n)
        spcov_result = spcov.spcov(
        init_omega_r,
        S_r,
        Lambda_r,
        step_size=0.1,
        nesterov = True,
        n_outer_steps = 100,
        n_inner_steps = 100,
        tol_outer = 1e-04,
        thr_inner = 0.01,
        backtracking = 0.2,
        trace = 0
        )

        omega_hat_spcov = spcov_result[1]
        sigma_hat_spcov = np.dot(np.dot(np.dot(np.dot(X_transpose_X_inv,X_transpose),omega_hat_spcov), X_with_intercept),X_transpose_X_inv)

        # Step 10: Construct the t-statistics for beta_hat
        t_stat_ols = beta_hat / np.sqrt(np.diag(sigma_hat_ols))
        t_stat_robust = beta_hat / np.sqrt(np.diag(sigma_hat_robust))
        t_stat_spcov = beta_hat / np.sqrt(np.diag(sigma_hat_spcov))

        print(f"Iteration {i + 1} completed.")
        
        return beta_hat, t_stat_ols, t_stat_robust, t_stat_spcov
    finally:
        # Close the local R context
        ro.r("gc()")

In [14]:
# Initialize arrays to store results
all_beta_hat = []
all_t_stat_ols = []
all_t_stat_robust = []
all_t_stat_spcov = []
iteration = 0

# Define loop parameters
num_iterations = 20  # Number of iterations
num_processes = mp.cpu_count()  # Number of processes to run in parallel

In [15]:
if __name__ == "__main__":
    # Create a thread pool executor
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_processes) as executor:
        # Use a list comprehension to submit tasks to the thread pool
        futures = [executor.submit(run_iteration, i) for i in range(num_iterations)]
        
        # Retrieve results as they become available
        results = [future.result() for future in concurrent.futures.as_completed(futures)]
    
# Convert the results list to arrays
all_beta_hat = np.array([result[0] for result in results])
all_t_stat_ols = np.array([result[1] for result in results])
all_t_stat_robust = np.array([result[2] for result in results])
all_t_stat_spcov = np.array([result[3] for result in results])

# Now you can analyze or summarize the results as needed
average_beta_hat = np.mean(all_beta_hat, axis=0)
average_t_stat_ols = np.mean(all_t_stat_ols, axis=0)
average_t_stat_robust = np.mean(all_t_stat_robust, axis=0)
average_t_stat_spcov = np.mean(all_t_stat_spcov, axis=0)

print("Average beta_hat:", average_beta_hat)
print("Average t_stat_ols:", average_t_stat_ols)
print("Average t_stat_robust:", average_t_stat_robust)
print("Average t_stat_spcov:", average_t_stat_spcov)

Iteration 3 started...
Iteration 2 started...
Iteration 7 started...
Iteration 6 started...
Iteration 10 started...
Iteration 5 started...
Iteration 8 started...
Iteration 1 started...
Iteration 9 started...
Iteration 4 started...
Iteration 4 completed.
Iteration 10 completed.Iteration 6 completed.

Iteration 1 completed.
Iteration 3 completed.
Iteration 9 completed.
Iteration 5 completed.
Iteration 8 completed.
Iteration 2 completed.
Iteration 7 completed.
Iteration 11 started...
Iteration 16 started...
Iteration 15 started...
Iteration 17 started...
Iteration 14 started...
Iteration 12 started...
Iteration 18 started...
Iteration 16 completed.
Iteration 11 completed.
Iteration 13 started...
Iteration 17 completed.
Iteration 14 completed.
Iteration 20 started...
Iteration 12 completed.
Iteration 19 started...
Iteration 18 completed.
Iteration 15 completed.
Iteration 13 completed.
Iteration 20 completed.
Iteration 19 completed.
Average beta_hat: [-9.78623925e-16  1.91310763e+00 -2.5357